## Retrieval-Augmented Generation (RAG)

#### **Retrieval-Augmented Generation (RAG)** is a technique where an AI model retrieves relevant information from an external knowledge source before generating an answer. Instead of relying only on what it was trained on, the model searches documents, databases, or embeddings to ground its response in accurate, up-to-date information. For example, if a user asks, “Explain Python dataclasses,” the system first retrieves documents about dataclasses, then uses those to craft a response. This reduces hallucinations and improves factual correctness. RAG is especially useful for chatbots, support systems, and any application that needs precise domain knowledge. In short, RAG combines search with generation to produce more reliable answers.


InputQuery--> Relavant information for External Sources Such as Vector DB --> Prompts --> LLM ---> Format the output

For this process, we need vector DB with relavant knowledge, LLMs(We already have them from different providers), now lets configure vector DB. 

#### What is vector DB? 

A **vector database** is a specialized database designed to store and search high-dimensional vectors—numerical representations of text, images, audio, or other data. It enables fast similarity search, helping find items that are “closest” in meaning using algorithms like nearest-neighbor search. Vector DBs are commonly used in AI applications such as RAG, recommendation systems, and semantic search. In simple terms, they help computers find information that is similar in meaning, not just exact matches.


### Embeddings: Way to convert real world data into Numbers

**Embeddings** are numerical representations of data, like text, images, or audio, in a high-dimensional space. They capture the meaning or features of the data so that similar items are close together in this space. For example, in text embeddings, sentences with similar meaning will have vectors that are near each other. Embeddings are used in search, recommendation systems, and machine learning because they allow computers to measure similarity efficiently. Essentially, embeddings convert complex data into numbers that AI can understand and compare.

In our case, it is textual information. we used embedding to convert text into vextors.

In [1]:
#import necessary libraries.
import os
#os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")import warnings
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [2]:
#import embeeding models.
from langchain_huggingface import HuggingFaceEmbeddings
hug_embeds=HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")

from langchain_openai import OpenAIEmbeddings
#https://openai.com/index/new-embedding-models-and-api-updates/
openai_embeds = OpenAIEmbeddings(model="text-embedding-3-large")

In [3]:
#lets test simple sentece
query="Apple is a fruit"
query_embeds=hug_embeds.embed_query(query)
len(query_embeds)

384

In [5]:
query_embeds=openai_embeds.embed_query(query)
len(query_embeds)

3072

In [6]:
#you change the len of the vector by passing the vector dimention
openai_embeds = OpenAIEmbeddings(model="text-embedding-3-large",dimensions='384')
query_embeds=hug_embeds.embed_query(query)
len(query_embeds)

384

### Simple Similarity search 

In [7]:
#for simple demo for finding the relavant documetns to user query, take 5 documetns
simple_docs=['Human are great', 'Cat is animal', 'Computer is a machine','Banana is a long','Apple is sweet']
simple_doc_mebeds=hug_embeds.embed_documents(simple_docs)
len(simple_doc_mebeds)

5

In [8]:
import numpy as np
np.array(simple_doc_mebeds).shape

(5, 384)

In [9]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
cosine_similarity([query_embeds],simple_doc_mebeds)

array([[0.14793217, 0.21895693, 0.31491134, 0.51137612, 0.71066439]])

In [10]:
euclidean_distances([query_embeds],simple_doc_mebeds)

array([[1.30542536, 1.24983441, 1.17054567, 0.98855836, 0.7607044 ]])

This is a simple sentquery and 5 documents with less sentences but in real time there will be thousands of documents, GB of textual informtion. To process this level data, LangChain has the TextLoaders

### Simple Similarity Search Using Vector Stores using LangChain

### FAISS (Facebook AI similarity Search)

FAISS is a tool with algorithm that helps computers quickly find similar things in large sets of data. It works by turning information like text or images into numbers called vectors, then finding which ones are closest to each other. People use FAISS for things like search engines, recommendations, and AI applications that need fast similarity matching.

Key Parameters in Any Vector DB
- Vector Dimension
    - The number of elements in each vector (e.g., 768 for many text embeddings).
    - All vectors in the database must have the same dimension.
    - Higher dimensions can capture more information but may slow down search.
- Index Type
    - Flat or / brute-force: simple but slow for large datasets.
    - IVF (Inverted File Index): partitions vectors into clusters for faster search.
    - HNSW (Hierarchical Navigable Small World graphs): very fast approximate nearest neighbor search. (Clustering approach)
- Distance / Similarity Metric
    - Cosine similarity: measures angle between vectors (common for text).
    - Euclidean distance (L2): straight-line distance in space.
- Storage Permission: 
    - Whether vectors are stored in memory (fast) or on disk (scalable) or Cloud and Some vector DBs allow hybrid approaches for large datasets.
- Metadata / Filtering Options
    - Many vector DBs allow storing associated metadata with vectors.
    - Enables filtering results based on conditions, e.g., only search vectors where category="news
- Batch Size / Insertion Parameters
    - How many vectors are inserted at a time.
    - Impacts indexing speed and database efficiency.- 


In [11]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

#setting the indexing type
index=faiss.IndexFlatL2(384)

#### In memory vector server

In [12]:
vector_store=FAISS(embedding_function=hug_embeds,index=index,docstore=InMemoryDocstore(), index_to_docstore_id={})

In [13]:
vector_store.add_texts(simple_docs)

['c972d7b1-2878-4630-a70d-a2c77a8e97b0',
 '00d28b7d-5c26-42bb-8720-13e5b5a5ea57',
 '7b78c9b6-35ae-49d8-928f-a77f15f0f9ec',
 '608e55d7-0419-4855-970c-85af5c6b182a',
 '091d7e33-a3e4-43f7-9010-efed6e6ea2fc']

In [14]:
vector_store.index_to_docstore_id

{0: 'c972d7b1-2878-4630-a70d-a2c77a8e97b0',
 1: '00d28b7d-5c26-42bb-8720-13e5b5a5ea57',
 2: '7b78c9b6-35ae-49d8-928f-a77f15f0f9ec',
 3: '608e55d7-0419-4855-970c-85af5c6b182a',
 4: '091d7e33-a3e4-43f7-9010-efed6e6ea2fc'}

In [15]:
vector_store.docstore.search(vector_store.index_to_docstore_id[1])

Document(id='00d28b7d-5c26-42bb-8720-13e5b5a5ea57', metadata={}, page_content='Cat is animal')

In [16]:
vector_store.similarity_search(query,k=2)

[Document(id='091d7e33-a3e4-43f7-9010-efed6e6ea2fc', metadata={}, page_content='Apple is sweet'),
 Document(id='608e55d7-0419-4855-970c-85af5c6b182a', metadata={}, page_content='Banana is a long')]

This is a simple sentquery and 5 documents with less sentences but in real time there will be thousands of documents, GB of textual informtion. To process this level data, LangChain has the TextLoaders and TextLoader prepares the documents and the

### Similarity Search or Retriver for Large Documents using LangChain

In [17]:
from langchain_community.document_loaders import TextLoader
text_loader=TextLoader('./data/human_evolution.txt')
documents=text_loader.load()
documents

[Document(metadata={'source': './data/human_evolution.txt'}, page_content='Human evolution is the long biological and cultural process through which modern humans emerged from earlier hominin ancestors. Humanity’s story begins with early primates that adapted to life in trees and later transitioned to mixed forest and savanna environments. Early hominins developed bipedalism, which freed their hands for tool use and expanded their ability to travel efficiently. Over millions of years, selective pressures shaped changes in anatomy, behavior, and cognition. Species such as Australopithecus afarensis demonstrate early upright walking while still retaining features useful for climbing. The emergence of Homo habilis marked a shift toward more sophisticated stone toolmaking and improved problem-solving abilities. Homo erectus expanded these trends by developing larger brain sizes, better tools, and the ability to control fire. Fire use allowed for cooking, which increased caloric availabilit

the document can contain long senteces, paragraphs withoug line breaks. it will be hard to feed all the data together. Hence, textual information has to splitted for better processing

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=25)
split_documents=text_splitter.split_documents(documents)
split_documents

[Document(metadata={'source': './data/human_evolution.txt'}, page_content='Human evolution is the long biological and cultural process through which modern humans emerged from earlier hominin ancestors. Humanity’s story begins with early primates that adapted to life in'),
 Document(metadata={'source': './data/human_evolution.txt'}, page_content='that adapted to life in trees and later transitioned to mixed forest and savanna environments. Early hominins developed bipedalism, which freed their hands for tool use and expanded their ability to'),
 Document(metadata={'source': './data/human_evolution.txt'}, page_content='their ability to travel efficiently. Over millions of years, selective pressures shaped changes in anatomy, behavior, and cognition. Species such as Australopithecus afarensis demonstrate early'),
 Document(metadata={'source': './data/human_evolution.txt'}, page_content='demonstrate early upright walking while still retaining features useful for climbing. The emergence of

In [19]:
index1=faiss.IndexFlatL2(384)
vector_store_docs=FAISS(embedding_function=hug_embeds,index=index1,docstore=InMemoryDocstore(),index_to_docstore_id={})

In [20]:
vector_store_docs.add_documents(documents=split_documents)

['d77ca5d2-ad17-48a0-b236-dac6ff41b5e2',
 'c5c120c2-e1c4-41fa-b977-cfb6ce14add2',
 '7ffc004e-5341-486d-a793-0023947ebd91',
 'c697e792-7e6e-47ed-8334-270d6046ab4c',
 '165eaaf4-362c-4eb3-8810-7f72f822c3be',
 'af7641bc-e6cd-46f5-a29e-329c1ac81b4e',
 'c3ccd117-4234-47a6-84ee-fcaaf13c5c42',
 '8ccb5142-60d8-4959-974f-3c5149e4c8ae',
 '62ca01e8-d769-4ea1-bc8a-1f02fb7489e7',
 '5ec026c6-99ac-43d8-ae7a-6e8e15f4ff88',
 'fccc4d56-8fdb-409e-b81c-c91e20d488c2',
 'd60cbfa4-ff2a-4ba1-a6bb-1e2b0a7d8645',
 'af1c8b19-69d9-468b-b5ef-1fee16d5a4ef',
 'c262e91f-7a85-4bbe-9050-ee323580729f',
 '32147bb7-b914-43e1-876b-7c550a9e71ef',
 'be0055c4-204f-412a-8a5c-f0a540d07d12',
 '8f12eadc-23a0-48ee-85b7-57b0e5bca5e5',
 '0f8dd71d-16ba-4a87-a47a-f0e7ac9aef35',
 '85976ddf-bf6d-4373-a2d1-a2e7d2dae59c',
 'b67fca5c-9c3f-458f-b9a1-05ff17ac9a01',
 '849d188d-9701-4beb-afc6-f1720db55242',
 '277ae968-ba1e-4430-8d8f-5c7c444cb4e6',
 'a068ee2e-91eb-482f-a8dc-c2c91c94e272',
 'cdaf04cf-1ca3-450c-bff9-4e332aebca71',
 '73d76ed2-4a41-

In [ ]:
# print("FAISS index size:", vector_store.index.ntotal)
# print("Docstore size:", len(vector_store.docstore._dict))

FAISS index size: 55
Docstore size: 25


In [22]:
vector_store_docs.similarity_search(query= "What is human evolution interm of enineering ?", k=4 )

[Document(id='d77ca5d2-ad17-48a0-b236-dac6ff41b5e2', metadata={'source': './data/human_evolution.txt'}, page_content='Human evolution is the long biological and cultural process through which modern humans emerged from earlier hominin ancestors. Humanity’s story begins with early primates that adapted to life in'),
 Document(id='32147bb7-b914-43e1-876b-7c550a9e71ef', metadata={'source': './data/human_evolution.txt'}, page_content='Technological advances such as metalworking, navigation, and engineering reshaped human interactions with the environment. Cultural evolution began to outpace biological evolution, allowing rapid'),
 Document(id='277ae968-ba1e-4430-8d8f-5c7c444cb4e6', metadata={'source': './data/human_evolution.txt'}, page_content='change, now influence the future direction of human adaptation. Biological evolution continues, though cultural and technological evolution shape human lives more strongly. Today’s humans remain part'),
 Document(id='a068ee2e-91eb-482f-a8dc-c2c91c9

In [23]:
#another way to get the same result using retriever functionality
retriever=vector_store_docs.as_retriever(search_kwargs={'k': 4})
retriever.invoke(input="What is human evolution interm of enineering ?")

[Document(id='d77ca5d2-ad17-48a0-b236-dac6ff41b5e2', metadata={'source': './data/human_evolution.txt'}, page_content='Human evolution is the long biological and cultural process through which modern humans emerged from earlier hominin ancestors. Humanity’s story begins with early primates that adapted to life in'),
 Document(id='32147bb7-b914-43e1-876b-7c550a9e71ef', metadata={'source': './data/human_evolution.txt'}, page_content='Technological advances such as metalworking, navigation, and engineering reshaped human interactions with the environment. Cultural evolution began to outpace biological evolution, allowing rapid'),
 Document(id='277ae968-ba1e-4430-8d8f-5c7c444cb4e6', metadata={'source': './data/human_evolution.txt'}, page_content='change, now influence the future direction of human adaptation. Biological evolution continues, though cultural and technological evolution shape human lives more strongly. Today’s humans remain part'),
 Document(id='a068ee2e-91eb-482f-a8dc-c2c91c9

#### Ondisk Vector Store

In [ ]:
#store to local disk
vector_store_docs.save_local('./data/faiss/humanevolution')

In [26]:
vector_store_docs_ondisk=FAISS.load_local("./data/faiss/humanevolution",embeddings=hug_embeds,allow_dangerous_deserialization=True)
vector_store_docs_ondisk.similarity_search(query= "What is human evolution interm of enineering ?", k=4)

[Document(id='d77ca5d2-ad17-48a0-b236-dac6ff41b5e2', metadata={'source': './data/human_evolution.txt'}, page_content='Human evolution is the long biological and cultural process through which modern humans emerged from earlier hominin ancestors. Humanity’s story begins with early primates that adapted to life in'),
 Document(id='32147bb7-b914-43e1-876b-7c550a9e71ef', metadata={'source': './data/human_evolution.txt'}, page_content='Technological advances such as metalworking, navigation, and engineering reshaped human interactions with the environment. Cultural evolution began to outpace biological evolution, allowing rapid'),
 Document(id='277ae968-ba1e-4430-8d8f-5c7c444cb4e6', metadata={'source': './data/human_evolution.txt'}, page_content='change, now influence the future direction of human adaptation. Biological evolution continues, though cultural and technological evolution shape human lives more strongly. Today’s humans remain part'),
 Document(id='a068ee2e-91eb-482f-a8dc-c2c91c9

so far, we have build vector stores, able to get the relavant information. Now lets combinite them together and pass to llm to build RAG.

#### Cloud Vector DB

### Pinecone

Pinecone is a managed vector database service that helps store and search high-dimensional embeddings efficiently. It handles indexing, scaling, and fast similarity search so developers don’t have to manage complex infrastructure. Pinecone is widely used in AI applications like RAG, semantic search, and recommendation systems.


In [52]:
pinecone_api_key=os.getenv("PINECONE_API_KEY")
from pinecone import Pinecone
from pinecone import ServerlessSpec #Serverless: Server will be Managed by the cloud provider

#Step 1: Create Index on cloud
pc_cloud=Pinecone(api_key=pinecone_api_key)
index_name="agenticai"
#creating a index
if not pc_cloud.has_index(index_name):
    pc_cloud.create_index(name=index_name,dimension=768,metric="cosine",spec=ServerlessSpec(cloud="aws",region="us-east-1")    
)

#Step2 User PC index in Langchain
from langchain_pinecone import PineconeVectorStore
pc_vector_store= PineconeVectorStore(index=pc_cloud.Index(index_name),embedding=hug_embeds)

In [ ]:
pc_vector_store.add_documents(documents=split_documents)


['62d56f6f-9d8c-4f87-8147-ae0803e62708',
 'f59883e2-5bd6-43d2-898d-76199942763a',
 'ea751f5a-cd0f-4fb0-9077-354cadda8a05',
 '53e99533-8070-4579-bea6-042b61958dc0',
 '098ba6cd-9edd-44cf-818d-6fc11db122a9',
 '76a329a4-3307-48e4-b3ce-dfc23c18af62',
 'dc0d9c4a-128f-446c-954c-c3433b9ddaa6',
 '0a4ccded-602d-4742-a02f-ede1adcaeac6',
 'a6884e42-aca7-4731-bd6b-00400e58d73f',
 'b652ee27-25b7-4736-bf7c-e89d9f7b9ff4',
 'aac4ba96-a3a6-4566-a1ed-126f03d9b72f',
 '9bfb610f-8ae5-4beb-af51-d728e07752ae',
 'ec9efe43-1898-4055-acee-1353fd5e0e11',
 '75b1e70c-9e6d-4ea0-9bc5-1ae82e0d1524',
 '77e20cc6-851d-43d4-bbc8-0c658fb46d1a',
 '3600d8b4-3386-47e0-a8f8-32712fb19b20',
 '69546d69-bc2e-42c1-ab58-2aba5216b5df',
 'a4db5d8b-d9e5-4fc7-9db2-035ec361bb41',
 'b118043a-2106-43e3-802c-f0a0cc1269cc',
 '24f2d6df-e281-4f28-96cf-176bdd58b1c8',
 '541cf5d5-d9c2-4a6f-aa46-c58526437709',
 '26a48c7f-9de4-4864-a96a-c424d90a751d',
 'b4a3d9de-0963-4739-a74e-0a0e0a4e3c6d',
 'd39a41cd-fe43-4833-9d3a-e2bdbede5421',
 '8650f171-9d50-

In [54]:
pc_vector_store.similarity_search(query= "What is human evolution interm of enineering ?", k=4 )

[Document(id='62d56f6f-9d8c-4f87-8147-ae0803e62708', metadata={'source': './data/human_evolution.txt'}, page_content='Human evolution is the long biological and cultural process through which modern humans emerged from earlier hominin ancestors. Humanity’s story begins with early primates that adapted to life in'),
 Document(id='77e20cc6-851d-43d4-bbc8-0c658fb46d1a', metadata={'source': './data/human_evolution.txt'}, page_content='Technological advances such as metalworking, navigation, and engineering reshaped human interactions with the environment. Cultural evolution began to outpace biological evolution, allowing rapid'),
 Document(id='b4a3d9de-0963-4739-a74e-0a0e0a4e3c6d', metadata={'source': './data/human_evolution.txt'}, page_content='humans remain part of the natural world while also transforming it on an unprecedented scale. The story of human evolution highlights resilience, creativity, and continual adaptation. It reveals how'),
 Document(id='26a48c7f-9de4-4864-a96a-c424d90a

In [55]:
#another way to get the same result using retriever functionality
retriever=pc_vector_store.as_retriever(search_kwargs={'k': 4})
retriever.invoke(input="What is human evolution interm of enineering ?")

[Document(id='62d56f6f-9d8c-4f87-8147-ae0803e62708', metadata={'source': './data/human_evolution.txt'}, page_content='Human evolution is the long biological and cultural process through which modern humans emerged from earlier hominin ancestors. Humanity’s story begins with early primates that adapted to life in'),
 Document(id='77e20cc6-851d-43d4-bbc8-0c658fb46d1a', metadata={'source': './data/human_evolution.txt'}, page_content='Technological advances such as metalworking, navigation, and engineering reshaped human interactions with the environment. Cultural evolution began to outpace biological evolution, allowing rapid'),
 Document(id='26a48c7f-9de4-4864-a96a-c424d90a751d', metadata={'source': './data/human_evolution.txt'}, page_content='change, now influence the future direction of human adaptation. Biological evolution continues, though cultural and technological evolution shape human lives more strongly. Today’s humans remain part'),
 Document(id='b4a3d9de-0963-4739-a74e-0a0e0a4

### Build RAG Pipeline

In [56]:

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="o3-mini")

In [37]:
### format the documents. 
from langchain_core.prompts import ChatPromptTemplate
prompt= ChatPromptTemplate.from_template("Write 5 sentences about {query}")
chain=prompt|llm|StrOutputParser()

In [38]:
chain.invoke({"query":"What is human evolution interms of enineering ?"})

'1. From an engineering perspective, human evolution can be seen as nature’s own iterative design process, where each genetic modification acts like a component upgrade to improve overall performance.  \n2. Natural selection, much like an engineering feedback loop, fine-tunes the human body by selecting traits that optimize function, resilience, and adaptability in diverse environments.  \n3. Over millennia, the evolution of structural features such as bipedalism, complex hand dexterity, and advanced brain function reflects a series of refinements similar to engineering breakthroughs.  \n4. Modern biotechnology and genetic engineering increasingly allow us to mimic and enhance these natural evolutionary processes, leading to innovations in medicine and biotechnology.  \n5. By studying human evolution through an engineering lens, scientists can better understand the mechanisms of improvement and apply those lessons to design technologies that enhance human capability and quality of life

In [ ]:
retriever=vector_store_docs.as_retriever(search_kwargs={'k': 4})
#context_docs=retriever.invoke(input="What is human evolution interm of enineering ?")
def format_docs(context_docs):
    return "\n\n".join(doc.page_content for doc in context_docs)

'Human evolution is the long biological and cultural process through which modern humans emerged from earlier hominin ancestors. Humanity’s story begins with early primates that adapted to life in\n\nTechnological advances such as metalworking, navigation, and engineering reshaped human interactions with the environment. Cultural evolution began to outpace biological evolution, allowing rapid\n\nchange, now influence the future direction of human adaptation. Biological evolution continues, though cultural and technological evolution shape human lives more strongly. Today’s humans remain part\n\nhumans remain part of the natural world while also transforming it on an unprecedented scale. The story of human evolution highlights resilience, creativity, and continual adaptation. It reveals how'

In [41]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [47]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [48]:
rag_chain.invoke("What is human evolution interms of enineering ?")

'Human evolution in terms of engineering refers to how human innovation—ranging from early toolmaking to modern technology—has shaped our adaptation to and manipulation of the environment. Engineering advances such as metalworking, navigation, and infrastructure have allowed societies to overcome challenges and rapidly evolve culturally and technologically. This interplay of engineering and human adaptation has become a defining feature of our species’ ongoing evolution.'